<h1>
<center>
Practice Midterm 2
</center>
</h1>
<div class=h1_cell>
<p>
The Practice and Actual midterm will focus on weeks 6 and 7. In particular, you will be asked to import your week7 library. You can still work on the week6 piece of this practice midterm but will need to finish week7 homework to do it all.
<p>
  I'll use the shelter table here. The actual midterm will use another dataset.
</div>

In [0]:
import pandas as pd
shelter_table = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT44JuI4VWv1ZOov2Gz7ZlMZk4scUkI5xXbwGQwK455Ue7a-jy77KZ_olLf6JQyBl7RjeNmF2KOIiwE/pub?gid=1192502698&single=true&output=csv')
shelter_table.head(1)

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,adopted,n/s,mix,type_Cat,type_Dog,no_name
0,A671945,Hambone,2/12/2014 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White,0,1,1,0,1,0


In [0]:
len(shelter_table)

1000

In [0]:
#Note that if you are not done with week 7 homework, you can load week6b instead and work on the first part on random forests

!rm library_w19_week7.py

rm: cannot remove 'library_w19_week7.py': No such file or directory


In [0]:
from google.colab import files
files.upload()

Saving library_w19_week7.py to library_w19_week7.py


{'library_w19_week7.py': b'import pandas as pd\nimport numpy as np\nfrom functools import reduce\n\ndef predictor_case(row, pred, target):\n\tcase_dict = {(0,0): \'true_negative\', (1,1): \'true_positive\', (0,1): \'false_negative\', (1,0): \'false_positive\'}\n\tactual = row[target]\n\tprediction = row[pred]\n\tcase = case_dict[(prediction, actual)]\n\treturn case\n\ndef accuracy(cases):\n    tp = cases[\'true_positive\'] if \'true_positive\' in cases else 0\n    tn = cases[\'true_negative\'] if \'true_negative\' in cases else 0\n    fp = cases[\'false_positive\'] if \'false_positive\' in cases else 0\n    fn = cases[\'false_negative\'] if \'false_negative\' in cases else 0\n\n    result = (tp + tn)/(tp+tn+fp+fn) if (tp+tn+fp+fn) != 0 else 0\n    return result\n\ndef f1(cases):\n    #the heart of the matrix\n    tp = cases[\'true_positive\'] if \'true_positive\' in cases else 0\n    tn = cases[\'true_negative\'] if \'true_negative\' in cases else 0\n    fp = cases[\'false_positive\'] 

In [0]:
from library_w19_week7 import *
%who function

accuracy	 all_distances	 build_pred	 build_tree_iter	 compute_prediction	 compute_training	 euclidean_distance	 f1	 find_best_splitter	 
forest_builder	 forest_scores	 generate_table	 gig	 gini	 informedness	 k_fold	 k_fold_random	 knn	 
oob_table	 oob_vote_taker	 path_id	 predictor_case	 probabilities	 produce_scores	 reorder_paths	 row_to_vect	 tree_predictor	 
verify_unique	 vote_taker	 


In [0]:
pd.set_option('display.max_columns', None)


<h2>Columns used in all questions</h2>

We will use 5 columns from the table to make predictions on `adopted`.

In [0]:
usable_columns = [
 'n/s', 'mix', 'type_Cat', 'type_Dog', 'no_name'
]

target = 'adopted'

<h1>
Part 1. Explore random forest behavior
</h1>
<div class=h1_cell>
<p>
For prediction purposes, we have been treating a forest as a single predictor. It does call on its trees to get a prediction, but we don't see that. All we see is the final prediction. I'd like to dig deeper into individual tree behavior. I'll ask you to complete a set of programming problems to do this.
</div>

<h2>
Problem 1. Build a matrix of tree predictions
</h2>
<div class=h1_cell>
<p>
The matrix will be a list of lists. Let's say the forest we are working with has 11 trees. Then the first list will be the predictions of the 11 trees for row 0 of the shelter table, i.e., it will be a list of 11 binary values. The next list will be the 11 predictions for row 1 of the shelter table. Given that we have 1000 rows in the shelter table, we will have 1000 lists in our outer list.
<p>
Let's say I store my matrix in `all_trees`. Then `all_trees[i][j]` will represent the vote of the jth tree for row i.
  <p>
    Here is what you should see for the first 10 rows.
<p>
<img src='https://www.dropbox.com/s/zwde42yc2z2lq4j/Screenshot%202019-02-18%2009.44.33.png?raw=1'>

</div>

<h2>
Here is forest to test on
</h2>
<div class=h1_cell>
<p>
  But first set random seeds so you get same results as mine.
</div>

In [0]:
import random

rng = np.random.RandomState(24)  #Will pass as arg to pandas sample method
random.seed(2000)

In [0]:
forest11 = forest_builder(shelter_table, usable_columns, target, hypers={'total-trees':11, 'random-state':rng})
len(forest11)

11

[[0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0],
 [1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1],
 [0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0],
 [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1]]

<h2>
Problem 2: find the rows where all trees have the same answer
</h2>
<div class=h1_cell>
<p>
Right or wrong, they are unanimous. Place your answer in `unan_rows`.

</div>

In [0]:
unan_rows = unanimous_rows(all_trees)

In [0]:
len(unan_rows)

244

In [0]:
print(unan_rows)

[3, 5, 7, 10, 16, 17, 20, 31, 35, 37, 39, 41, 43, 47, 49, 50, 52, 55, 57, 64, 67, 68, 76, 77, 81, 83, 87, 88, 98, 102, 103, 113, 114, 115, 117, 123, 127, 137, 138, 147, 149, 156, 157, 158, 162, 164, 170, 179, 184, 187, 199, 200, 210, 214, 216, 218, 221, 230, 233, 239, 244, 246, 248, 256, 268, 272, 273, 277, 279, 280, 283, 285, 286, 294, 300, 312, 313, 318, 321, 326, 330, 336, 337, 339, 341, 343, 345, 347, 351, 356, 363, 366, 370, 382, 385, 387, 388, 391, 393, 399, 409, 418, 423, 428, 433, 439, 442, 443, 446, 448, 449, 454, 456, 458, 459, 464, 468, 473, 474, 478, 483, 487, 491, 493, 497, 498, 500, 505, 507, 508, 509, 511, 517, 522, 525, 527, 542, 558, 559, 562, 563, 566, 570, 576, 581, 587, 589, 593, 599, 600, 602, 603, 604, 609, 612, 613, 617, 619, 623, 625, 630, 632, 642, 643, 646, 647, 658, 666, 672, 676, 682, 700, 707, 712, 713, 722, 724, 726, 732, 738, 741, 745, 750, 751, 753, 754, 760, 761, 766, 768, 775, 781, 785, 787, 798, 802, 803, 804, 806, 808, 809, 811, 813, 818, 829, 831, 8

<h2>
Problem 3: find the trees that give the same answer for all rows
</h2>
<div class=h1_cell>
<p>
Which trees, if any, always produce the same prediction. Place your answer in `constant_trees`. The values in the list are tree indices.
</div>

In [0]:
nc = never_change(all_trees)  #no trees are constant
nc

[]

<h2>Problem 4: razor-thin majority</h2>

Find the rows that are decided by a bare majority. In the case of 11 trees, 6 vote one way and 5 the other. So majority of only 1 vote.

In [0]:
print(razor_list)
len(razor_list)  #142

[4, 9, 22, 33, 45, 59, 72, 75, 78, 80, 85, 91, 92, 95, 111, 121, 125, 132, 134, 139, 153, 165, 172, 174, 177, 178, 180, 186, 190, 192, 194, 211, 212, 225, 227, 229, 231, 232, 245, 249, 252, 254, 257, 260, 265, 271, 274, 278, 288, 293, 304, 320, 327, 335, 338, 344, 349, 354, 355, 357, 360, 365, 373, 378, 384, 389, 394, 396, 404, 407, 408, 414, 422, 441, 451, 467, 479, 501, 502, 514, 519, 520, 523, 533, 539, 541, 543, 554, 556, 569, 588, 591, 601, 605, 607, 629, 637, 640, 654, 673, 679, 706, 721, 723, 731, 743, 748, 762, 773, 774, 777, 786, 789, 797, 810, 812, 816, 817, 839, 848, 858, 864, 866, 868, 872, 873, 885, 897, 900, 909, 914, 919, 921, 927, 935, 937, 945, 947, 963, 972, 985, 989]


142

<h1>Part 2. Explore K-NN behavior</h1>

Once you have finished week 7 homework, you can take these on.
<p>
  Note: you may want to slice off the first 100 rows of the Shelter table for debugging. Remember that KNN runs slow. Once you are happy with results on small table you can try on full table. Up to you.

<h2>
Problem 5: K-NN voting weights
</h2>
<div class=h1_cell>
<p>
Normally the top K rows in K-NN have an equal vote. Let's modify this, I want a new voting scheme as follows:
  <pre>
  if a row in top-k is within 0 distance from the target row, give it 4 votes.
  if a row in top-k is within 1 distance from the target row, give it 3 votes.
  if a row in top-k is within 1.5 distance from the target row, give it 2 votes.
  else give it 1 vote
  </pre>
  The row gets its top score. So a row within distance 0 gets exactly 4 votes.
  <p>
    Add the votes up as normal to get a prediction, ties go to prediction of 0.
</div>

<h2>Here are my results with k=10</h2>

When I compute accuracy with no scaeld-voting, i.e., as normal, I get .621 accuracy.
<p>
When I compute accuracy with my new scaled-voting scheme, I get .596 accuracy.

In [0]:
all_predictions_w = []
all_predictions_r = []
for i in range(len(shelter_table)):
  all_predictions_w.append(knn2(i, shelter_table, usable_columns, 10, target))
  all_predictions_r.append(knn(i, shelter_table, usable_columns, 10, target))
  

In [0]:
z = list(zip(all_predictions_w, shelter_table[target]))
correct = z.count((0,0)) + z.count((1,1))
correct/len(shelter_table)

0.596

In [0]:
z = list(zip(all_predictions_r, shelter_table[target]))
correct = z.count((0,0)) + z.count((1,1))
correct/len(shelter_table)

0.621

<h2>
Problem 6: Determine how many times each row was correct
</h2>
<div class=h1_cell>
<p>


In [0]:

sorted(participant_pairs, key=lambda pair: pair[1], reverse=True)

[(0, 168),
 (13, 168),
 (14, 168),
 (19, 168),
 (24, 168),
 (3, 136),
 (7, 136),
 (10, 136),
 (16, 136),
 (37, 136),
 (39, 136),
 (41, 136),
 (43, 136),
 (47, 136),
 (50, 134),
 (15, 130),
 (28, 130),
 (2, 128),
 (8, 128),
 (12, 128),
 (23, 128),
 (26, 128),
 (40, 127),
 (44, 127),
 (51, 127),
 (22, 65),
 (59, 65),
 (78, 65),
 (80, 65),
 (91, 65),
 (95, 65),
 (1, 60),
 (21, 60),
 (29, 60),
 (38, 60),
 (46, 58),
 (33, 54),
 (45, 54),
 (75, 54),
 (92, 54),
 (17, 42),
 (20, 40),
 (31, 40),
 (114, 40),
 (117, 40),
 (147, 40),
 (199, 40),
 (214, 40),
 (230, 40),
 (248, 40),
 (6, 39),
 (18, 39),
 (25, 39),
 (42, 39),
 (60, 39),
 (62, 39),
 (73, 39),
 (82, 39),
 (118, 39),
 (35, 35),
 (57, 35),
 (88, 35),
 (210, 35),
 (283, 35),
 (312, 35),
 (370, 35),
 (439, 35),
 (446, 35),
 (32, 25),
 (89, 25),
 (97, 25),
 (140, 25),
 (141, 25),
 (166, 25),
 (11, 24),
 (99, 24),
 (122, 24),
 (146, 24),
 (191, 24),
 (202, 24),
 (215, 24),
 (226, 24),
 (5, 21),
 (30, 14),
 (67, 14),
 (79, 14),
 (195, 14),
 (

<h2>Problem 7: K-NN how many voting blocks</h2>
  
  I'd like to know how often pairs of rows in the top-k vote the same. So if k is 3, I will have 3 rows in top-3. Call them r1, r2, r3. I would check the following:
  
1. if (r1,r2) vote the same then I record that.
2. if (r1,r3) vote the same then I record that.
3. if (r2,r3) vote the same then I record that.

I do that for the entire table. What I whould have is a list or dictionary of counts across the entire table. Final I thing I want is sorted list. Here is first part of sorted list I got for k = 10.
<pre>
[((2, 23), 296),
 ((2, 26), 296),
 ((0, 13), 296),
 ((0, 14), 296),
 ((0, 19), 296),
 ((0, 24), 296),
 ((2, 12), 294),
 ((8, 26), 288),
 ((1, 29), 190),
 ((1, 38), 190),
 ((1, 21), 189),
 ((1, 46), 185),
 ((3, 39), 136),
 ((3, 41), 136),
 ((3, 43), 136),
 ((3, 47), 136),
 ((7, 50), 134),
 ((3, 50), 134),
 ((3, 37), 133),
 ((7, 47), 129),
 ((6, 118), 46),
 ((6, 82), 40),
 ((4, 229), 19),
 ((4, 304), 19),
 ((4, 5), 14),
 ((5, 449), 13),
 ((5, 478), 13),
 ((5, 517), 13),
</pre>

In [0]:

sorted(block_pairs, key=lambda pair: pair[1], reverse=True)

[((2, 23), 296),
 ((2, 26), 296),
 ((0, 13), 296),
 ((0, 14), 296),
 ((0, 19), 296),
 ((0, 24), 296),
 ((2, 12), 294),
 ((8, 26), 288),
 ((1, 29), 190),
 ((1, 38), 190),
 ((1, 21), 189),
 ((1, 46), 185),
 ((3, 39), 136),
 ((3, 41), 136),
 ((3, 43), 136),
 ((3, 47), 136),
 ((7, 50), 134),
 ((3, 50), 134),
 ((3, 37), 133),
 ((7, 47), 129),
 ((6, 118), 46),
 ((6, 82), 40),
 ((4, 229), 19),
 ((4, 304), 19),
 ((4, 5), 14),
 ((5, 449), 13),
 ((5, 478), 13),
 ((5, 517), 13),
 ((2, 27), 9),
 ((8, 27), 9),
 ((7, 52), 9),
 ((3, 52), 9),
 ((6, 169), 9),
 ((5, 642), 9),
 ((5, 442), 8),
 ((4, 4), 7),
 ((4, 17), 7),
 ((5, 5), 7),
 ((5, 17), 7),
 ((4, 893), 6),
 ((1, 1), 5),
 ((1, 996), 4),
 ((3, 16), 2),
 ((7, 43), 2),
 ((1, 285), 2),
 ((4, 738), 1)]